# Mask and visualize a GeoTiff

This notebook takes the examples [mask geotiff](../scala/mask_geotiff.ipynb) written in Scala and [plot geotiffs](../python/plot_GeoTiffs.ipynb) written in Python and merges them into a single notebook using [Pixiedust](https://github.com/ibm-watson-data-lab/pixiedust).

In [1]:
import pixiedust

Pixiedust database opened successfully


## Python

### Dependencies

In [2]:
#Add all dependencies to PYTHON_PATH
import sys
sys.path.append("/usr/lib/spark/python")
sys.path.append("/usr/lib/spark/python/lib/py4j-0.10.4-src.zip")
sys.path.append("/usr/lib/python3/dist-packages")

#Define environment variables
import os
os.environ["HADOOP_CONF_DIR"] = "/etc/hadoop/conf"
os.environ["PYSPARK_PYTHON"] = "python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "ipython"

#Load PySpark to connect to a Spark cluster
from pyspark import SparkConf, SparkContext

#To read GeoTiffs as a ByteArray
from io import BytesIO
import matplotlib.pyplot as plt
import rasterio
from rasterio import plot
from rasterio.io import MemoryFile

### Shared variables

In [3]:
mask_path = "hdfs:///user/hadoop/usa_mask.tif"
geo_path = "hdfs:///user/hadoop/spring-index/BloomFinal/1980.tif"
masked_geotiff = "hdfs:///user/emma/spring-index/BloomFinal/1980_mask.tif"
tmp_geo = "/tmp/1980_mask_clone.tif"

In [33]:
sc._conf.set('spark.jars',  'file:/data/local/pixiedust/bin/cloudant-spark-v2.0.0-185.jar:/usr/lib/spark/jars/*')

In [34]:
sc._conf.getAll()

[('spark.driver.extraClassPath', '/data/local/pixiedust/data/libs/*'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.dynamicAllocation.initialExecutors', '2'),
 ('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false'),
 ('spark.jars.ivy', ''),
 ('spark.eventLog.dir', '/data/local/spark/spark-events'),
 ('spark.hadoop.fs.s3a.fs.s3a.fast.upload', 'true'),
 ('spark.app.id', 'local-1499677300081'),
 ('spark.hadoop.fs.s3a.endpoint', 'http://145.100.59.64:9091'),
 ('spark.jars.packages', ''),
 ('spark.hadoop.fs.s3a.access.key', 'A24H1RIGV4RKFGXJTEMS'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.local.dir', '/data/local/spark/tmp/'),
 ('spark.memory.storageFraction', '0.5'),
 ('spark.daemon.memory', '3g'),
 ('spark.kryoserializer.buffer', '512m'),
 ('spark.worker.memory', '14g'),
 ('spark.driver.memory', '8g'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.default.parallelism', '32'),
 ('spark.driver.port', '37392'),
 ('spark.submit.deployMode', 'c

## Scala

Here is the scala code to mask a GeoTiff.

### Dependencies

In [35]:
%%scala
import sys.process._
import geotrellis.proj4.CRS
import geotrellis.raster.io.geotiff.writer.GeoTiffWriter
import geotrellis.raster.io.geotiff.{SinglebandGeoTiff, _}
import geotrellis.raster.{CellType, DoubleArrayTile}
import geotrellis.spark.io.hadoop._
import geotrellis.vector.{Extent, ProjectedExtent}
import org.apache.spark.mllib.linalg.Vector
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}

import spire.syntax.cfor._


var geo_num_cols_rows :(Int, Int) = (0, 0)
val geo_tiles_RDD = sc.hadoopGeoTiffRDD(geo_path).values

val geo_extents_withIndex = sc.hadoopMultibandGeoTiffRDD(geo_path).keys.zipWithIndex().map{case (e,v) => (v,e)}
var geo_projected_extent = (geo_extents_withIndex.filter(m => m._1 == 0).values.collect())//(0)
var geo_projected = geo_projected_extent.take(10)

Py4JJavaError: An error occurred while calling o1668.invoke.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 8, localhost, executor driver): java.lang.ClassCastException: cannot assign instance of scala.collection.immutable.List$SerializationProxy to field org.apache.spark.rdd.RDD.org$apache$spark$rdd$RDD$$dependencies_ of type scala.collection.Seq in instance of org.apache.spark.rdd.MapPartitionsRDD
	at java.io.ObjectStreamClass$FieldReflector.setObjFieldValues(ObjectStreamClass.java:2133)
	at java.io.ObjectStreamClass.setObjFieldValues(ObjectStreamClass.java:1305)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2251)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2169)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2027)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1535)
	at java.io.Objec

In [ ]:


val geo_tiles_withIndex = geo_tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
val geo_tile0 = (geo_tiles_withIndex.filter(m => m._1==0).values.collect())(0)
geo_num_cols_rows = (geo_tile0.cols, geo_tile0.rows)
val geo_cellT = geo_tile0.cellType

### Read a GeoTiff file

In [8]:
%%scala
var geo_projected_extent = new ProjectedExtent(new Extent(0,0,0,0), CRS.fromName("EPSG:3857"))
var geo_num_cols_rows :(Int, Int) = (0, 0)
val geo_tiles_RDD = sc.hadoopGeoTiffRDD(geo_path).values

val geo_extents_withIndex = sc.hadoopMultibandGeoTiffRDD(geo_path).keys.zipWithIndex().map{case (e,v) => (v,e)}
geo_projected_extent = (geo_extents_withIndex.filter(m => m._1 == 0).values.collect())(0)

val geo_tiles_withIndex = geo_tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
val geo_tile0 = (geo_tiles_withIndex.filter(m => m._1==0).values.collect())(0)
geo_num_cols_rows = (geo_tile0.cols, geo_tile0.rows)
val geo_cellT = geo_tile0.cellType

b'pixiedustRunner.scala:32: error: not found: type ProjectedExtent'
b'        var geo_projected_extent = new ProjectedExtent(new Extent(0,0,0,0), CRS.fromName("EPSG:3857"))'
b'                                       ^'
b'pixiedustRunner.scala:32: error: not found: type Extent'
b'        var geo_projected_extent = new ProjectedExtent(new Extent(0,0,0,0), CRS.fromName("EPSG:3857"))'
b'                                                           ^'
b'pixiedustRunner.scala:32: error: not found: value CRS'
b'        var geo_projected_extent = new ProjectedExtent(new Extent(0,0,0,0), CRS.fromName("EPSG:3857"))'
b'                                                                            ^'
b'pixiedustRunner.scala:34: error: value hadoopGeoTiffRDD is not a member of org.apache.spark.SparkContext'
b'val geo_tiles_RDD = sc.hadoopGeoTiffRDD(geo_path).values'
b'                       ^'
b'pixiedustRunner.scala:36: error: value hadoopMultibandGeoTiffRDD is not a member of org.apache.spark.SparkConte

### Read Mask

In [10]:
%%scala
val mask_tiles_RDD = sc.hadoopGeoTiffRDD(mask_path).values
val mask_tiles_withIndex = mask_tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
val mask_tile0 = (mask_tiles_withIndex.filter(m => m._1==0).values.collect())(0)

b'pixiedustRunner.scala:32: error: value hadoopGeoTiffRDD is not a member of org.apache.spark.SparkContext'
b'        val mask_tiles_RDD = sc.hadoopGeoTiffRDD(mask_path).values'
b'                                ^'
b'one error found'


### Mask GeoTiff

In [ ]:
%%scala
val res_tile = geo_tile0.localInverseMask(mask_tile0, 1, 0).toArrayDouble()

### Save the new GeoTiff file

In [ ]:
%%scala
val clone_tile = DoubleArrayTile(res_tile, geo_num_cols_rows._1, geo_num_cols_rows._2)

val cloned = geotrellis.raster.DoubleArrayTile.empty(geo_num_cols_rows._1, geo_num_cols_rows._2)
cfor(0)(_ < geo_num_cols_rows._1, _ + 1) { col =>
    cfor(0)(_ < geo_num_cols_rows._2, _ + 1) { row =>
        val v = clone_tile.getDouble(col, row)
        cloned.setDouble(col, row, v)
    }
}

val geoTif = new SinglebandGeoTiff(cloned, geo_projected_extent.extent, geo_projected_extent.crs, Tags.empty, GeoTiffOptions.DEFAULT)

//Save GeoTiff to /tmp
GeoTiffWriter.write(geoTif, tmp_geo)

//Upload to HDFS
var cmd = "hadoop dfs -copyFromLocal -f " + tmp_geo + " " + masked_geo
Process(cmd)!

cmd = "rm -fr " + tmp_geo
Process(cmd)!

## Python

It will read the masked GeoTiff created using Scala and plot it using **matplotlib**

### Plot masked GeoTiff

In [ ]:
#Read it
data = sc.binaryFiles(masked_geo).take(1)
dataByteArray = bytearray(data[0][1])

#Lets check if the files was read correctly by printing its metadata
with MemoryFile(dataByteArray) as memfile:
    with memfile.open() as dataset:
        print(dataset.profile)

%matplotlib notebook
with MemoryFile(dataByteArray) as memfile:
    with memfile.open() as dataset:
        plot.show((dataset,1))